In [1]:
import pandas as pd            # Data handling
import numpy as np             # Numerical operations
import torch                   # PyTorch core
import torch.nn as nn          # Neural network modules
import random
from torch.utils.data import DataLoader, TensorDataset, random_split, Dataset
from sklearn.model_selection import train_test_split  # Data splitting
from sklearn.preprocessing import MinMaxScaler       # Scaling
import matplotlib.pyplot as plt  # (Optional) plotting results
from sklearn.metrics import mean_squared_error
import os
import joblib
import optuna
import torch.optim as optim
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

C:\Users\user\anaconda3\envs\PyTorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Check for GPU availability and set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# --- Step 1: Load the CSV file ---
#csv_file = 'deflection_with_temperatures_final.csv'  # your CSV file
csv_file = 'dataset.csv'  # your CSV file
df = pd.read_csv(csv_file)
df = df.drop(columns=["Time"])  

In [4]:
input_feature_suffixes = ["x2", "y2", "x6", "y6", "x13", "y13", "x29", "y29", "x31", "y31", "x33", "y33", "T3", "T8", "T12", "T14", "T16", "T21", "T24", "T26", "T27"]  # 21 names
output_feature_suffixes = ['x2', 'y2', 'y3', 'TS3', 'x6', 'y6',  'x7', 'y7', 'y8', 'TS8', 'x11', 'y11', 'y12', 'TS12', 'x13', 'y13', 'y14', 'TS14', 'y16', 'TS16', 'x17', 
                           'y17', 'x19', 'y19', 'y21', 'TS21', 'x22', 'y22', 'y24', 'TS24', 'x25', 'y25', 'y26', 'TS26',  'y27', 'TS27', 'x29', 'y29', 'x31', 'y31', 'x33', 'y33',"T3", "T8", "T12", "T14", "T16", "T21", "T24", "T26", "T27"]  # 42+9 names

In [5]:
# --- Step 3: Parse data case by case ---
X_cases = []
y_cases = []

num_cases = 2400
timesteps = 181

for i in range(num_cases):
    prefix = f'case{i+1}_'
    
    input_cols = [prefix + feat for feat in input_feature_suffixes]
    output_cols = [prefix + feat for feat in output_feature_suffixes]

    # --- Defensive check: ensure all columns exist ---
    missing_inputs = [col for col in input_cols if col not in df.columns]
    missing_outputs = [col for col in output_cols if col not in df.columns]
    if missing_inputs or missing_outputs:
        raise ValueError(f"Missing columns in case {i+1}: {missing_inputs + missing_outputs}")
    
    X_case = df[input_cols].values  # shape: (181, 21)
    y_case = df[output_cols].values  # shape: (181, 51)

    X_cases.append(X_case)
    y_cases.append(y_case)

# --- Step 4: Stack into NumPy arrays ---
X_data = np.stack(X_cases)  # (2400, 181, 21)
y_data = np.stack(y_cases)  # (2400, 181, 51)

print(X_data.shape, y_data.shape)

(2400, 181, 21) (2400, 181, 51)


In [6]:
# Flatten along cases (keep time series)
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_data, y_data, test_size=0.15, random_state=42)

# Then split train+val into train and val
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.23529, random_state=42)  # 0.1765 of 85% = 15%

# Final shapes:
print("X_train:", X_train.shape)
print("X_val:", X_val.shape)
print("X_test:", X_test.shape)

print("y_train:", y_train.shape)
print("y_val:", y_val.shape)
print("y_test:", y_test.shape)

X_train: (1560, 181, 21)
X_val: (480, 181, 21)
X_test: (360, 181, 21)
y_train: (1560, 181, 51)
y_val: (480, 181, 51)
y_test: (360, 181, 51)


In [7]:
X_test[44][180]

array([ 37.4391  ,  -5.35601 ,   5.14348 ,  -5.85926 ,   0.326028,
        -5.83046 , -37.4451  ,  -5.35659 ,  -5.14957 ,  -5.85983 ,
        -0.332374,  -5.83102 ,  25.      ,  20.      ,  25.      ,
        20.      ,  20.      ,  20.      ,  25.      ,  20.      ,
        20.      ])

In [8]:
# ---- Flatten the training data for fitting the scaler ----
X_train_2d = X_train.reshape(-1, X_train.shape[2])  # shape: (samples * timesteps, features) → (181 * num_cases, 21)
y_train_2d = y_train.reshape(-1, y_train.shape[2])  # shape: (samples * timesteps, outputs) → (181 * num_cases, 51)

# ---- Fit scalers on training data only ----
X_scaler = MinMaxScaler()
X_train_norm_2d = X_scaler.fit_transform(X_train_2d)

y_scaler = MinMaxScaler()
y_train_norm_2d = y_scaler.fit_transform(y_train_2d)

# ---- Apply the same scaler to val and test data ----
X_val_norm_2d = X_scaler.transform(X_val.reshape(-1, X_val.shape[2]))
X_test_norm_2d = X_scaler.transform(X_test.reshape(-1, X_test.shape[2]))

y_val_norm_2d = y_scaler.transform(y_val.reshape(-1, y_val.shape[2]))
y_test_norm_2d = y_scaler.transform(y_test.reshape(-1, y_test.shape[2]))

# ---- Reshape back to original shape ----
X_train_norm = X_train_norm_2d.reshape(X_train.shape)
X_val_norm = X_val_norm_2d.reshape(X_val.shape)
X_test_norm = X_test_norm_2d.reshape(X_test.shape)

y_train_norm = y_train_norm_2d.reshape(y_train.shape)
y_val_norm = y_val_norm_2d.reshape(y_val.shape)
y_test_norm = y_test_norm_2d.reshape(y_test.shape)

In [9]:
print("y_train shape:", y_train.shape)

y_train shape: (1560, 181, 51)


In [10]:
X_test_norm.shape

(360, 181, 21)

In [11]:
# Save the scalers
joblib.dump(X_scaler, 'X_scaler.save')
joblib.dump(y_scaler, 'y_scaler.save')

['y_scaler.save']

In [12]:
# Denormalize X_test_norm (must be 2D for inverse_transform)
#X_test_flat = X_test_norm.reshape(-1, X_test_norm.shape[-1])  # shape: (360*181, 21)

# Inverse transform
#X_test_denorm_flat = X_scaler.inverse_transform(X_test_flat)  # shape: (360*181, 21)

# Reshape back to original 3D shape
#X_test_denorm = X_test_denorm_flat.reshape(X_test_norm.shape) 

#X_test_denorm[359][180]

In [13]:
def create_sliding_windows(X, y, window_size=30, target_size=1):
    X_seq = []
    y_seq = []

    num_cases, num_steps, num_features = X.shape

    for case in range(num_cases):
        for t in range(num_steps - window_size - target_size + 1):
            X_window = X[case, t : t + window_size, :]                # shape: (30, 21)
            y_target = y[case, t + window_size : t + window_size + target_size, :]  # shape: (1, 51)
            X_seq.append(X_window)
            y_seq.append(y_target.squeeze(0))  # make shape (51,) instead of (1,51)

    return np.array(X_seq), np.array(y_seq)

# Create sliding windows
X_train_sliding, y_train_sliding = create_sliding_windows(X_train_norm, y_train_norm, 30, 1)
X_val_sliding, y_val_sliding = create_sliding_windows(X_val_norm, y_val_norm, 30, 1)
X_test_sliding, y_test_sliding = create_sliding_windows(X_test_norm, y_test_norm, 30, 1)

# Print shapes
print("X_train_sliding:", X_train_sliding.shape)  # (N, 30, 21)
print("y_train_sliding:", y_train_sliding.shape)  # (N, 51)

X_train_sliding: (235560, 30, 21)
y_train_sliding: (235560, 51)


In [14]:
class SlidingWindowDataset(Dataset):
    def __init__(self, X, Y):
        """
        X: np.array of shape (N, window_size, in_dim)  e.g. (N, 30, 21)
        Y: np.array of shape (N, out_dim)              e.g. (N, 51)
        """
        assert X.shape[0] == Y.shape[0]
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        # returns: (30,21), (51,)
        return self.X[idx], self.Y[idx]


In [15]:
batch_size = 32  # or whatever you prefer

# Train sliding dataset (optional: for your old teacher-forcing model / metrics)
train_dataset_sliding = SlidingWindowDataset(X_train_sliding, y_train_sliding)

# Validation and test sliding datasets
val_dataset_sliding   = SlidingWindowDataset(X_val_sliding,   y_val_sliding)
test_dataset_sliding  = SlidingWindowDataset(X_test_sliding,  y_test_sliding)

# DataLoaders
train_loader_sliding = DataLoader(train_dataset_sliding, batch_size=batch_size, shuffle=True)
val_loader_sliding   = DataLoader(val_dataset_sliding,   batch_size=batch_size, shuffle=False)
test_loader_sliding  = DataLoader(test_dataset_sliding,  batch_size=batch_size, shuffle=False)

print("Sliding-window loaders:")
print("  train batches:", len(train_loader_sliding))
print("  val batches:",   len(val_loader_sliding))
print("  test batches:",  len(test_loader_sliding))

Sliding-window loaders:
  train batches: 7362
  val batches: 2265
  test batches: 1699


In [16]:
class FullSequenceDataset(Dataset):
    def __init__(self, X, Y):
        """
        X: np.array of shape (N_cases, T, in_dim)  e.g. (N, 181, 21)
        Y: np.array of shape (N_cases, T, out_dim) e.g. (N, 181, 51)
        """
        assert X.shape[0] == Y.shape[0]
        assert X.shape[1] == Y.shape[1]
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()
    
    def __len__(self):
        return self.X.shape[0]
    
    def __getitem__(self, idx):
        # returns full sequence for one case
        return self.X[idx], self.Y[idx]   # shapes: (T, 21), (T, 51)

# Create datasets
train_dataset_full = FullSequenceDataset(X_train_norm, y_train_norm)
val_dataset_full   = FullSequenceDataset(X_val_norm,   y_val_norm)   # optional, if needed
test_dataset_full  = FullSequenceDataset(X_test_norm,  y_test_norm)  # optional

# DataLoader for training with scheduled sampling
batch_size = 64  # tune if needed
train_loader_full = DataLoader(train_dataset_full, batch_size=batch_size, shuffle=True)

print("Train full-sequence batches:", len(train_loader_full))

Train full-sequence batches: 49


In [17]:
# Build mapping from suffix -> index in the 51-dim output
suffix_to_out_idx = {suf: i for i, suf in enumerate(output_feature_suffixes)}

# Get indices for the 21 input features in the same order
idx_in_list = [suffix_to_out_idx[suf] for suf in input_feature_suffixes]
print("idx_in_list:", idx_in_list)

# Convert to tensor (later moved to device)
idx_in = torch.tensor(idx_in_list, dtype=torch.long)

idx_in_list: [0, 1, 4, 5, 14, 15, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [18]:
def train_model_scheduled(
    model,
    params,
    train_loader_full,
    val_loader_sliding,
    device,
    idx_in,
    max_epochs=2000,
    window_size=30,
    rollout_horizon=8,
):
    # ---- Loss function ----
    if params['loss_function'] == 'mse':
        criterion = nn.MSELoss()
    elif params['loss_function'] == 'huber':
        criterion = nn.HuberLoss(delta=params.get('huber_delta', 1.0))
    elif params['loss_function'] == 'mae':
        criterion = nn.L1Loss()
    else:
        criterion = nn.MSELoss()
    
    # ---- Optimizer ----
    if params['optimizer'] == 'adam':
        optimizer = optim.Adam(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
    elif params['optimizer'] == 'adamw':
        optimizer = optim.AdamW(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
    elif params['optimizer'] == 'rmsprop':
        optimizer = optim.RMSprop(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
    else:
        optimizer = optim.AdamW(
            model.parameters(),
            lr=params['learning_rate'],
            weight_decay=params['weight_decay']
        )
    
    # ---- Scheduler ----
    if params['scheduler'] == 'plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=10
        )
    elif params['scheduler'] == 'cosine':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=max_epochs, eta_min=1e-6
        )
    elif params['scheduler'] == 'step':
        scheduler = optim.lr_scheduler.StepLR(
            optimizer, step_size=30, gamma=0.5
        )
    else:
        scheduler = None
    
    grad_clip = params.get('grad_clip', None)
    
    best_val_loss = float('inf')
    best_state = None
    patience = 2000
    patience_counter = 0

    # ============================
    # NEW: loss history containers
    # ============================
    train_losses = []
    val_losses   = []
    
    # move idx_in to device once
    idx_in = idx_in.to(device)
    
    for epoch in range(1, max_epochs + 1):
        model.train()
        train_loss = 0.0
        
        # ---- scheduled sampling probability ----
        import math
        center = max_epochs / 2
        sigma  = max_epochs / 6
        max_pred_prob = 0.7

        x = epoch
        use_pred_prob = max_pred_prob * math.exp(
            - ((x - center) ** 2) / (2 * sigma ** 2)
        )
        
        for X_seq, Y_seq in train_loader_full:
            X_seq = X_seq.to(device)
            Y_seq = Y_seq.to(device)
            B, T, _ = X_seq.shape
            
            w = window_size
            h = rollout_horizon
            
            max_start = T - w - h
            if max_start <= 0:
                raise ValueError("Sequence too short for given window_size and rollout_horizon.")
            
            s = random.randint(0, max_start)
            
            optimizer.zero_grad()
            batch_loss = 0.0
            
            window_in = X_seq[:, s:s+w, :]
            
            for step in range(h):
                y_pred = model(window_in)
                y_true = Y_seq[:, s + w + step, :]
                
                batch_loss = batch_loss + criterion(y_pred, y_true)
                
                x_true_next = y_true[:, idx_in]
                x_pred_next = y_pred[:, idx_in]
                
                if use_pred_prob <= 0.0:
                    x_next = x_true_next
                elif use_pred_prob >= 1.0:
                    x_next = x_pred_next
                else:
                    mask = (torch.rand(B, 1, device=device) < use_pred_prob).float()
                    x_next = mask * x_pred_next + (1.0 - mask) * x_true_next
                
                x_next = x_next.unsqueeze(1)
                window_in = torch.cat([window_in, x_next], dim=1)[:, -w:, :]
            
            batch_loss = batch_loss / h
            batch_loss.backward()
            
            if grad_clip is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=grad_clip)
            
            optimizer.step()
            train_loss += batch_loss.item()
        
        avg_train_loss = train_loss / len(train_loader_full)
        
        # ---- Validation ----
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for X_batch, y_batch in val_loader_sliding:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)
                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item()
        
        avg_val_loss = val_loss / len(val_loader_sliding)

        # ============================
        # NEW: save losses per epoch
        # ============================
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        
        # Scheduler step
        if scheduler:
            if params['scheduler'] == 'plateau':
                scheduler.step(avg_val_loss)
            else:
                scheduler.step()
        
        print(
            f"Epoch [{epoch}/{max_epochs}] "
            f"Train Loss: {avg_train_loss:.6f} "
            f"Val Loss: {avg_val_loss:.6f} "
            f"use_pred_prob={use_pred_prob:.3f}"
        )
        
        # ---- Early stopping ----
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break
    
    # ---- load best weights ----
    if best_state is not None:
        model.load_state_dict(best_state)

    # ============================
    # NEW: save model & losses
    # ============================
    torch.save(
        {
            "train_losses": train_losses,
            "val_losses": val_losses,
            "best_val_loss": best_val_loss,
            "params": params,
        },
        "training_losses.pth"
    )

    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "params": params,
        },
        "final_trained_model.pth"
    )
    
    return best_val_loss, model

In [19]:
class OptimizedLSTM(nn.Module):
    def __init__(self, input_size=21, hidden_size=128, num_layers=3, 
                 output_size=51, dropout_p=0.2, activation='relu'):
        super(OptimizedLSTM, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm_layers = nn.ModuleList()
        
        # First LSTM layer
        self.lstm_layers.append(nn.LSTM(input_size, hidden_size, batch_first=True))
        
        # Additional LSTM layers
        for i in range(num_layers - 1):
            self.lstm_layers.append(nn.LSTM(hidden_size, hidden_size, batch_first=True))
        
        # Dropout
        self.dropout = nn.Dropout(dropout_p)
        
        # Activation function
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'gelu':
            self.activation = nn.GELU()
        elif activation == 'leaky_relu':
            self.activation = nn.LeakyReLU()
        else:
            self.activation = nn.ReLU()
        
        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, hidden_size // 2)
        self.fc2 = nn.Linear(hidden_size // 2, output_size)
        
    def forward(self, x):
        batch_size = x.size(0)
        
        # Pass through LSTM layers
        for i, lstm in enumerate(self.lstm_layers):
            out, _ = lstm(x)
            if i < len(self.lstm_layers) - 1:  # Don't apply dropout after last LSTM
                out = self.dropout(self.activation(out))
            x = out
        
        # Take the last time step
        out = out[:, -1, :]
        
        # Fully connected layers
        out = self.activation(self.fc1(out))
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

In [20]:
# latest 
best_params = {
    'hidden_size': 256,
    'num_layers': 3,
    'dropout': 0.118,
    'activation': 'leaky_relu',
    'learning_rate':0.002616,
    'weight_decay': 0.000026,
    'optimizer': 'adamw',
    'grad_clip': 1.8070018004980979,
    'loss_function': 'huber',
    'scheduler': 'cosine',
    'huber_delta': 1.31
}

In [ ]:
# Build idx_in from suffix lists
suffix_to_out_idx = {suf: i for i, suf in enumerate(output_feature_suffixes)}
idx_in_list = [suffix_to_out_idx[suf] for suf in input_feature_suffixes]
idx_in = torch.tensor(idx_in_list, dtype=torch.long)

# Create model with best_params
final_model = OptimizedLSTM(
    input_size=21,
    hidden_size=best_params['hidden_size'],
    num_layers=best_params['num_layers'],
    output_size=51,
    dropout_p=best_params['dropout'],
    activation=best_params['activation']
).to(device)

print("Training final model with scheduled sampling...")
final_val_loss, trained_model = train_model_scheduled(
    model=final_model,
    params=best_params,
    train_loader_full=train_loader_full,
    val_loader_sliding=val_loader_sliding,   # from your sliding-window step
    device=device,
    idx_in=idx_in,
    max_epochs=800,
    window_size=30,
    rollout_horizon=8
)

print("Best validation loss (scheduled sampling):", final_val_loss)

In [23]:
# Load saved loss history
checkpoint = torch.load("training_losses.pth", map_location="cpu")

train_losses = checkpoint["train_losses"]
val_losses   = checkpoint["val_losses"]

In [ ]:
plt.figure(figsize=(8, 6))

plt.plot(train_losses, label="Training Loss", linewidth=2)
plt.plot(val_losses, label="Validation Loss", linewidth=2)

plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [25]:
def evaluate_one_step_return_arrays(model, data_loader, device, loss_function='huber', huber_delta=1.0):
    model.eval()
    
    if loss_function == 'mse':
        criterion = nn.MSELoss()
    elif loss_function == 'huber':
        criterion = nn.HuberLoss(delta=huber_delta)
    elif loss_function == 'mae':
        criterion = nn.L1Loss()
    else:
        criterion = nn.MSELoss()

    all_preds = []
    all_trues = []
    total_loss = 0.0

    with torch.no_grad():
        for X_batch, y_batch in data_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()

            all_preds.append(y_pred.cpu().numpy())
            all_trues.append(y_batch.cpu().numpy())

    avg_loss = total_loss / len(data_loader)
    all_preds = np.concatenate(all_preds, axis=0)   # (N,51)
    all_trues = np.concatenate(all_trues, axis=0)   # (N,51)

    mae = mean_absolute_error(all_trues, all_preds)
    mse = mean_squared_error(all_trues, all_preds)
    rmse = np.sqrt(mse)
    r2 = r2_score(all_trues, all_preds)

    print("\n=== One-step EVAL (teacher-forcing) ===")
    print(f"Loss ({loss_function}): {avg_loss:.6f}")
    print(f"MAE:   {mae:.6f}")
    print(f"MSE:   {mse:.6f}")
    print(f"RMSE:  {rmse:.6f}")
    print(f"R²:    {r2:.4f}")
    
    return avg_loss, mae, rmse, r2, all_trues, all_preds

In [26]:
avg_loss, mae, rmse, r2, y_true_norm, y_pred_norm = evaluate_one_step_return_arrays(
    trained_model,
    test_loader_sliding,
    device,
    loss_function=best_params.get('loss_function', 'huber'),
    huber_delta=best_params.get('huber_delta', 1.0)
)



=== One-step EVAL (teacher-forcing) ===
Loss (huber): 0.000128
MAE:   0.006947
MSE:   0.000256
RMSE:  0.015997
R²:    0.9854


In [27]:
def per_feature_metrics(y_true, y_pred, feature_names=None):
    """
    y_true, y_pred: arrays of shape (N_samples, n_features)
    feature_names: optional list of length n_features
    """
    from sklearn.metrics import mean_squared_error, r2_score

    n_features = y_true.shape[1]
    rmses = []
    r2s = []

    for j in range(n_features):
        mse_j = mean_squared_error(y_true[:, j], y_pred[:, j])
        rmse_j = np.sqrt(mse_j)
        r2_j = r2_score(y_true[:, j], y_pred[:, j])
        rmses.append(rmse_j)
        r2s.append(r2_j)

    rmses = np.array(rmses)
    r2s = np.array(r2s)

    print("\nPer-feature RMSE and R² (normalized space):")
    for j in range(n_features):
        name = feature_names[j] if feature_names is not None else f"f{j}"
        print(f"{j:2d} ({name:8s}) → RMSE={rmses[j]:.4f}, R²={r2s[j]:.4f}")

    return rmses, r2s


In [28]:
avg_loss, mae, rmse, r2, y_true_norm, y_pred_norm = evaluate_one_step_return_arrays(
    trained_model,
    test_loader_sliding,
    device,
    loss_function=best_params.get('loss_function', 'huber'),
    huber_delta=best_params.get('huber_delta', 1.0)
)

# If you know the physical meaning of each of the 51 outputs:
feature_names = output_feature_suffixes  # length 51, from your earlier list

rmses_feat, r2_feat = per_feature_metrics(
    y_true_norm,
    y_pred_norm,
    feature_names=feature_names
)



=== One-step EVAL (teacher-forcing) ===
Loss (huber): 0.000128
MAE:   0.006947
MSE:   0.000256
RMSE:  0.015997
R²:    0.9854

Per-feature RMSE and R² (normalized space):
 0 (x2      ) → RMSE=0.0076, R²=0.9935
 1 (y2      ) → RMSE=0.0103, R²=0.9943
 2 (y3      ) → RMSE=0.0231, R²=0.9476
 3 (TS3     ) → RMSE=0.0270, R²=0.9897
 4 (x6      ) → RMSE=0.0065, R²=0.9931
 5 (y6      ) → RMSE=0.0098, R²=0.9963
 6 (x7      ) → RMSE=0.0059, R²=0.9888
 7 (y7      ) → RMSE=0.0149, R²=0.9702
 8 (y8      ) → RMSE=0.0243, R²=0.9564
 9 (TS8     ) → RMSE=0.0103, R²=0.9987
10 (x11     ) → RMSE=0.0051, R²=0.9903
11 (y11     ) → RMSE=0.0131, R²=0.9764
12 (y12     ) → RMSE=0.0304, R²=0.9288
13 (TS12    ) → RMSE=0.0110, R²=0.9987
14 (x13     ) → RMSE=0.0057, R²=0.9925
15 (y13     ) → RMSE=0.0095, R²=0.9958
16 (y14     ) → RMSE=0.0241, R²=0.9461
17 (TS14    ) → RMSE=0.0088, R²=0.9983
18 (y16     ) → RMSE=0.0264, R²=0.9654
19 (TS16    ) → RMSE=0.0110, R²=0.9989
20 (x17     ) → RMSE=0.0060, R²=0.9892
21 (y17   